# Imports

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import re

# Globals

These parameters can be changed as seen fit.

In [ ]:
PATH = "Cropped_Fern_Photos3"  # Change this to the folder with your images in it

COLOR_LOWER = (20, 40, 40)  # The lower threshold of hue, saturation, and brightness the computer picks up.
COLOR_UPPER = (70, 255, 255) # The upper threshold of hue, saturation, and brightness the computer picks up.

VISUALLY_VERIFY = True  # This toggle determines if the program will save the images
                        # of what the computer sees for you to manually look at.
VERIFY_PATH = "Verify"  # The folder name where you want to save the iamges for visually verifying.

CSV_FILE_NAME = "fern_data.csv" # The name the csv will be saved under.

# Hard coding
COLOR_LOWER2 = (20, 60, 40)

# Code

In [ ]:
# Create the information catchers
ID, Date, Group, Replicate, coverage_percent = [], [], [], [], []

# Loop over every image in the folder
for file in os.listdir(PATH):
    # Load in each image.
    img_file = os.path.join(PATH, file)
    img_bgr = cv2.imread(img_file)
    
    # Transform each image to RGB and HSV types.
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    
    # Mask each image according to lower and upper color thresholds.
    if file[:8] == "03032023" or file[:8] == "20032023":
        green_mask = cv2.inRange(img_hsv, COLOR_LOWER2, COLOR_UPPER)
    else:
        green_mask = cv2.inRange(img_hsv, COLOR_LOWER, COLOR_UPPER)
        
    # Finds the lost transparency. Important for calculating percentage.
    black_mask = cv2.inRange(img_rgb, (0,0,0), (0,0,0))
    
    # Gather all of the data
    ID.append(file)
    Date.append(file[:8])
    Group.append(file[8:-6])
    Replicate.append(file[-6])  # Only works if there are at most 9 replicates.
    coverage_percent.append(100 * np.sum(green_mask/255) / np.sum(np.invert(black_mask)/255))
    
    # Visually Verify 
    if VISUALLY_VERIFY:
        new_file_name = file[:-4] + "_verify" + file[-4:]
        new_file_path = os.path.join(VERIFY_PATH, new_file_name)
        
        # This code generates three side by side pictures 
        # demonstrating what the computer sees.
        
        temp_img = img_rgb.copy()
        result1 = cv2.bitwise_and(temp_img, temp_img, mask=np.invert(green_mask))
        
        temp_img = img_rgb.copy()
        result2 = cv2.bitwise_and(temp_img, temp_img, mask=green_mask)
        
        fig, axs = plt.subplots(1, 3, figsize=(15,5))
        axs[0].imshow(img_rgb)
        axs[1].imshow(result1)
        axs[2].imshow(result2)
        for ax in axs:
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False)
        fig.savefig(new_file_path, dpi=300)
        plt.close(fig)
    
# Convert the information into a dataframe.
data = pd.DataFrame({"ID":ID,
                      "Date":Date,
                      "Group":Group,
                      "Replicate":Replicate,
                      "% Fern Coverage":coverage_percent})
data['Date'] = pd.to_datetime(data['Date'], format='%d%m%Y')
data.to_csv(CSV_FILE_NAME, index=False)